In [13]:
from instruments_recognition.importing_data import *
from instruments_recognition.plotting import *
#import pymir.AudioFile

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [14]:
### Import transformed data (features) from file

energylbl1af = open('transformed_data/energylbl1a', 'rb')
energylbl1rf = open('transformed_data/energylbl1r', 'rb')
energylbl2af = open('transformed_data/energylbl2a', 'rb')
energylbl2rf = open('transformed_data/energylbl2r', 'rb')

energylbl1a_withmore = np.load(energylbl1af)
energylbl1r_withmore = np.load(energylbl1rf)
energylbl2a_withmore = np.load(energylbl2af)
energylbl2r_withmore = np.load(energylbl2rf)

#print(energylbl1a_withmore[0])

In [15]:
### LEARN 1


howmanyharmonics = 7

# forget the frequencies
# and forget the first volume which is always 1
def only_volumes(energylbl) :
    return list(map(lambda l : l[1][1:howmanyharmonics+1], energylbl))

# take only non nan from a list
def cleannan(l):
    return [x for x in l if not isnan(x)]

# input: list of lists
# output: only lists that do not contain any nans
def cleannan2(l):
    return [x for x in l if np.isfinite(x).all()]


energylbl1a = only_volumes(energylbl1a_withmore)
energylbl1r = only_volumes(energylbl1r_withmore)
energylbl2a = only_volumes(energylbl2a_withmore)
energylbl2r = only_volumes(energylbl2r_withmore)


# we can put l1 and l2, or just l2.
#energylbl1 = [ (l1 + l2) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + l2) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]
# it seems to work better with just l2
energylbl1 = energylbl1r
energylbl2 = energylbl2r


volumes1 = cleannan2(energylbl1)
volumes2 = cleannan2(energylbl2)
#shuffle the inputs (this is in place )
random.seed(5)
random.shuffle(volumes1)
random.shuffle(volumes2)


#nthvolume = [ map(lambda l : l[n], only_volumes) for n in range(0,num_harmonics) ]
#
#nthvolumecleaned = map(cleannan, nthvolume)
#
#averages = map(average, nthvolumecleaned)
#
#minusaverages = [ map(lambda m : m-averages[n], nthvolumecleaned[n]) for n in range(0,num_harmonics) ]
#
#variances = map(rms, minusaverages)
#
#variances, averages

#print(vd1[2])

In [16]:
### LEARN 2

from sklearn import neighbors, svm

#fractiontrain = 9./10.
fractiontrain = 5./6.
#fractiontrain = 4./5.
#fractiontrain = 1./100.

#training examples
trainvolumes1 = volumes1[:int(len(volumes1)*fractiontrain)]
trainvolumes2 = volumes2[:int(len(volumes2)*fractiontrain)]

howmanytrain1 = len(trainvolumes1)
howmanytrain2 = len(trainvolumes2)
howmanytrain = howmanytrain1 + howmanytrain2

#testing examples
testvolumes1 = volumes1[int(len(volumes1)*fractiontrain):len(volumes1)]
testvolumes2 = volumes2[int(len(volumes2)*fractiontrain):len(volumes2)]

howmanytest1 = len(testvolumes1)
howmanytest2 = len(testvolumes2)


training_ex = trainvolumes1 + trainvolumes2
labels = [ 1 for i in range(0,howmanytrain1) ] + [ 0 for i in range(0,howmanytrain2) ]


#########################

## assert: there are no testing samples in the training set

def is_in(e,l):
    res = False
    for v in l:
        if all([f1 == f2 for f1, f2 in zip(e,v)]) :
            res = True
    return res
            
#testing: is_in([1.],[[0.],[0.,4,3],[1.]])
    
alltests = testvolumes1 + testvolumes2

datasnooping = any([ is_in(test, training_ex) for test in alltests ])
if datasnooping :
    print("WARNING: you are testing on the training set")
print("doing data snooping? " + str(datasnooping))

doing data snooping? False


In [17]:
### Learning algorithm

## SVN does not seem to work
#clf = svm.SVC(kernel='linear')
#clf.fit(training_ex, labels) 


# nearest neighboor
n_neighbors = 5
clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(training_ex, labels) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [18]:
### COMPUTING ERROR

print("training with: " + str(howmanytrain))
print("testing with: " + str(howmanytest1 + howmanytest2))

goods1 = list(clf.predict(testvolumes1))
goods2 = [ 1-x for x in clf.predict(testvolumes2) ]

error = 1. - np.average(goods1+goods2)

print("error: " + str(error))


training with: 1021
testing with: 206
error: 0.18932038835


In [10]:
### IMPORT AND TRANSFORM DATA (this takes a while, you can import only the transfromed data above)

datadirectory1 = "./samples/flute/philharmonia/"
datadirectory2 = "./samples/oboe/philharmonia/"

howmanyexamples = 50

channels = 1

ex_wav1 = glob.glob(datadirectory1 + '*.wav')[:howmanyexamples]
ex_wav2 = glob.glob(datadirectory2 + '*.wav')[:howmanyexamples]

#ex_transformed1, freq_label1 = zip(*map(import_convert_transform, ex_wav1))
#ex_transformed2, freq_label2 = zip(*map(import_convert_transform, ex_wav2))

# zip(ex_transformed1, freq_label1)
# zip(ex_transformed2, freq_label2)

trandlbl1ar, vd1 = list(zip(*map(lambda e: import_convert_transform2(e,channels), ex_wav1)))
trandlbl2ar, vd2 = list(zip(*map(lambda e: import_convert_transform2(e,channels), ex_wav2)))

trandlbl1a, trandlbl1r = list(zip(*trandlbl1ar))
trandlbl2a, trandlbl2r = list(zip(*trandlbl2ar))


### LEARN 2

#print(trandlbl1a[0])



energylbl1a_withmore = [ harmonics_energy(ex_tr, freq_lbl) for ex_tr, freq_lbl in trandlbl1a ]
energylbl2a_withmore = [ harmonics_energy(ex_tr, freq_lbl) for ex_tr, freq_lbl in trandlbl2a ]

energylbl1r_withmore = [ harmonics_energy(ex_tr, freq_lbl) for ex_tr, freq_lbl in trandlbl1r ]
energylbl2r_withmore = [ harmonics_energy(ex_tr, freq_lbl) for ex_tr, freq_lbl in trandlbl2r ]


#### make a copy since we will shuffle the inputs and the shuffling works in place!
#energylbl1a_withmore = copy.deepcopy(energylbl1a_withmore_)
#energylbl2a_withmore = copy.deepcopy(energylbl2a_withmore_)
#energylbl1r_withmore = copy.deepcopy(energylbl1r_withmore_)
#energylbl2r_withmore = copy.deepcopy(energylbl2r_withmore_)


/home/lui/coding/ml/instruments/instruments_recognition/importing_data.py:188: RuntimeWarning: invalid value encountered in double_scalars
  # the total spectrum has len(ex) samples and the spectrum has 44100/2 frequencies


In [40]:
### Optional: save transformed data to disk

energylbl1af = open('transformed_data/energylbl1a', 'wb')
energylbl1rf = open('transformed_data/energylbl1r', 'wb')
energylbl2af = open('transformed_data/energylbl2a', 'wb')
energylbl2rf = open('transformed_data/energylbl2r', 'wb')

np.save(energylbl1af, energylbl1a_withmore)
np.save(energylbl1rf, energylbl1r_withmore)
np.save(energylbl2af, energylbl2a_withmore)
np.save(energylbl2rf, energylbl2r_withmore)

#print(energylbl1a_withmore[0])

energylbl1af.close()
energylbl1rf.close()
energylbl2af.close()
energylbl2rf.close()

#energylbl1af = open('transformed_data/energylbl1a', 'rb')
#
#energylbl1a_withmore = np.load(energylbl1af)
#
#print(energylbl1a_withmore[0])

In [608]:

def only_volumes1(energylbl) :
    return list(map(lambda l : l[1][1:], energylbl))

#only_volumes(energylbl1)

In [370]:
set([1,2]).intersection(set([1]))

{1}

In [467]:
a = [1,2,3,4,5,6]
np.random.seed(3)
np.random.shuffle(a)
a

[4, 6, 5, 2, 1, 3]

In [9]:
a = [1,2]
b = copy.copy(a)